In [1]:
import joblib
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#Load the dataset
df = pd.read_csv('email_data.csv')

#Data preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text'] = df['text'].apply(clean_text)

kazakh_stop_words = [
    'және', 'мен', 'де', 'да', 'бір', 'бар', 'болды', 'қалай', 'не', 'қандай',
    'еді', 'үшін', 'арқылы', 'менің', 'олардың', 'бұл', 'мұнда', 'онда',
    'сондықтан', 'ал', 'бірақ', 'сол', 'біз', 'сіз', 'ол', 'болу', 'еді', 'тағы',
    'осы', 'олар', 'бұл', 'ма', 'міне', 'менен', 'сізден', 'маған', 'сізге',
    'менің', 'сіздің', 'олардың', 'біздің', 'бізге', 'сіздерді', 'өз', 'өзі',
    'өзім', 'өзің', 'өзінің', 'өзіне', 'өздерің', 'өздеріне', 'өздеріміз',
    'өздеріңіздің', 'болады', 'болмайды', 'қазіргі', 'кейін', 'дейін', 'бұл',
    'осы', 'сол', 'онда', 'ондаған', 'мұнда', 'қазір', 'бірақ', 'сондықтан',
    'бәлкім', 'қажет', 'қандайда', 'бірде', 'қандай', 'жоқ', 'иә', 'жоқ', 'тек',
    'ғана', 'дегенмен', 'тағыда', 'болса', 'сіз', 'сіздер', 'олар', 'өзіміз',
    'өздеріңе', 'өзіңе', 'өздерін', 'бізбен', 'бізде', 'сіздерге', 'сіздермен',
    'сендер', 'оларға', 'олармен', 'мен', 'менімен', 'менде', 'мені', 'маған',
    'сіздермен', 'сіздерге', 'сіздерді', 'сіздерден', 'сіздермен', 'олармен',
    'оларға', 'олармен', 'оларды', 'олардың', 'олармен', 'олардан', 'оларға',
    'менде', 'мені', 'маған', 'бізбен', 'бізге', 'бізден', 'бізді', 'олармен',
    'оларды', 'олардан', 'оларға', 'олармен'
]

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(stop_words=kazakh_stop_words, max_features=150)
X = vectorizer.fit_transform(df['text'])

# Train-test split 20% test
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, random_state=42)

# Model training using Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Model evaluation
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print("Training Accuracy:", accuracy_score(y_train, y_pred_train))
print("Testing Accuracy:", accuracy_score(y_test, y_pred_test))

print("Classification Report on Training Data:\n", classification_report(y_train, y_pred_train))

print("Classification Report on Testing Data:\n", classification_report(y_test, y_pred_test))

# Save the model and vectorizer for later use
joblib.dump(model, 'kazakh_spam_classifier_rf_model.pkl')
joblib.dump(vectorizer, 'kazakh_tfidf_vectorizer.pkl')
print("Model and vectorizer saved successfully.")

Training Accuracy: 1.0
Testing Accuracy: 0.875
Classification Report on Training Data:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        45

    accuracy                           1.00        64
   macro avg       1.00      1.00      1.00        64
weighted avg       1.00      1.00      1.00        64

Classification Report on Testing Data:
               precision    recall  f1-score   support

           0       1.00      0.67      0.80         6
           1       0.83      1.00      0.91        10

    accuracy                           0.88        16
   macro avg       0.92      0.83      0.85        16
weighted avg       0.90      0.88      0.87        16

Model and vectorizer saved successfully.


In [2]:
import joblib

# Load the model and vectorizer
model = joblib.load('kazakh_spam_classifier_rf_model.pkl')
vectorizer = joblib.load('kazakh_tfidf_vectorizer.pkl')

def predict_spam(email_text):
    email_features = vectorizer.transform([email_text])
    prediction = model.predict(email_features)
    if prediction[0] == 1:
        return "Spam"
    else:
        return "Not Spam"

input_text = """Құрметті Қолданушы,

Сіз біздің арнайы акциямыздың жеңімпазы атандыңыз! Біз сіздің электрондық поштаңызды кездейсоқ таңдап, сізге ерекше жүлде беруді шештік. Жүлде ретінде сізге $10,000 ақшалай сыйлық немесе жаңа iPhone 14 беріледі!

Ұтысыңызды алу үшін, төмендегі сілтемеге өтіңіз және қажетті мәліметтерді енгізіңіз:
[Сілтеме: Ұтысыңызды алу үшін осы жерді басыңыз]

Жылдам әрекет етіңіз, себебі ұсыныс шектеулі мерзімге жарамды. Ұсыныс 24 сағат ішінде аяқталады.

Ұтысыңызды растау үшін төмендегі мәліметтерді толтырыңыз:

Атыңыз:
ЖСН:
Электрондық поштаңыз:
Телефон нөміріңіз:
Сілтеме арқылы тіркелгеннен кейін, біз сізбен хабарласамыз және жүлдеңізді қалай алуға болатынын түсіндіреміз. Егер сізде қандай да бір сұрақтар туындаса, бізге осы электрондық пошта арқылы жауап беріңіз.

Сіздің ақпаратыңыз қауіпсіздікте және құпия түрде сақталады."""

result = predict_spam(input_text)
print(f"The email is: {result}")

The email is: Spam
